# II. Deep Learning from scratch
## Convolutional Neural Networks for a cat classifer

## 1 - Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## 2. Convolutional Neural Networks
### 2.1 - Zero-Padding

In [2]:
def zero_pad(X, pad):
    """
    Arguments:
        X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
        pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    
    Returns:
        X_pad -- padded image of shape (m, n_H + 2*pad, n_W + 2*pad, n_C)
    """
    
    X_pad = np.pad(X, ((0,0), (pad,pad), (pad,pad), (0,0)), 'constant', constant_values=(0,))
    
    return X_pad

### 2.2 - Single step of convolution

In [8]:
def conv_single_step(a_slice_prev, W, b):
    """
    Arguments:
        a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
        W -- weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
        b -- bias parameters contained in a window - matrix of shape (1, 1, 1)
        
    Returns:
        Z -- a scalar value, result of convolving the sliding window (W, b) on a slice x of the input data
    """
    
    s = np.multiply(a_slice_prev, W)
    Z = np.sum(s)
    Z = np.add(Z, np.float(b))
    
    return Z

### 2.3 - Convolutional Neural Networks - Forward pass
$$ n_H = \lfloor \frac{n_{H_{prev}} - f + 2 \times pad}{stride} \rfloor +1$$
$$ n_W = \lfloor \frac{n_{W_{prev}} - f + 2 \times pad}{stride} \rfloor +1$$
$$ n_C = \text{number of filiters used in the convolution}$$

In [13]:
def conv_forward(A_prev, W, b, hparameters):
    """
    Arguments:
        A_prev -- output of activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
        W -- weight, numpy array of shape (f, f, n_C_prev, n_C)
        b -- biases, numpy array of shape (1, 1, 1, n_C)
        hparameters -- python dictionary containing "stride" and "pad"
    
    Returns:
        Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
        cache -- cache of values needed for the conv_backward() function
    """
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    (f, f, n_C_prev, n_C) = W.shape
    
    stride = hparameters["stride"]
    pad = hparameters["pad"]
    
    n_H = int((n_H_prev-f+2*pad)/stride)+1
    n_W = int((n_W_prev-f+2*pad)/stride)+1
    
    Z = np.zeros((m, n_H, n_W, n_C))
    
    A_prev_pad = zero_pad(A_prev, pad)
    
    for i in range(m):
        a_prev_pad = A_prev_pad[i,:,:,:]
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    
                    vert_start = stride*h
                    vert_end = vert_start+f
                    horiz_start = stride*w
                    horiz_end = horiz_start+f
                    
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end,:]
                    
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, W[:,:,:,c], b[:,:,:,c])
                    
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache

### 2.4 - Pooling layer
#### Forward Pooling
$$n_H = \lfloor \frac{n_{H_{prev}} - f}{stride} \rfloor + 1$$
$$n_W = \lfloor \frac{n_{W_{prev}} - f}{stride} \rfloor + 1$$
$$n_C = n_{C_{prev}}$$

In [26]:
def pool_forward(A_prev, hparameters, mode = "max"):
    """
    Arguments:
        A_prev -- input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
        hparameters -- python dictionary containing "f" and "stride"
        mode -- the pooling mode
        
    Returns:
        A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
        cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters
    """
    
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    n_H = int(1+(n_H_prev-f)/stride)
    n_W = int(1+(n_W_prev-f)/stride)
    n_C = n_C_prev
    
    A = np.zeros((m, n_H, n_W, n_C))
    
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    
                    vert_start = stride*h
                    vert_end = vert_start+f
                    horiz_start = stride*w
                    horiz_end = horiz_start+f
                    
                    a_prev_slice = A_prev[i,vert_start:vert_end, horiz_start:horiz_end,c]
                    
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_prev_slice)
    
    cache = (A_prev, hparameters)
    
    return A, cache

### 2.5 - Backpropagation in convolutional neural networks
#### a) Convolutional layer backward pass

In [27]:
np.random.seed(1)
A_prev = np.random.randn(2,4,4,3)

hparameters = {"stride":2, "f":3}

A, cache_conv = pool_forward(A_prev, hparameters)
print("A = ", A)
A, cache_conv = pool_forward(A_prev, hparameters, mode = "average")
print("A = ", A)

A =  [[[[1.74481176 0.86540763 1.13376944]]]


 [[[1.13162939 1.51981682 2.18557541]]]]
A =  [[[[ 0.02105773 -0.20328806 -0.40389855]]]


 [[[-0.22154621  0.51716526  0.48155844]]]]
